In [101]:
install.packages('distributions3')

library(tidyverse)
library(lubridate)
library(tidymodels)
library(distributions3)

## Functions for probability calculation

In [122]:
# Functions

# getProbabilities
    
getProbabilities = function(df, model, team, opponent, range, date) {
    
    print(date)
  
  fifa_date = test_date(date)
    
    print(fifa_date)
  
  dataFrame = data.frame(
    rank_opponent = getFifaRanking(opponent, fifa_date),
    Venue_H = ternary(1, 0, isNeutralAdjusted(team, opponent, 'World Cup 2022') == 'H'),
    Venue_N = ternary(1, 0, isNeutralAdjusted(team, opponent, 'World Cup 2022') == 'N'),
    Rank = getFifaRanking(team, fifa_date))
  
  lambda = predict(model, newdata = dataFrame, type = "response")
  
  p = Poisson(lambda)
  
  prob = pdf(p, range)
  
  return(prob)
}

# get probability data frame
getProbabilityMatrix = function(df, model, team, opponent, range, date) {
  
  res <- outer(getProbabilities(df, model, team, opponent, range, date), getProbabilities(df, model, opponent, team, range, date))
  df = data.frame(round(res, digits = 2))
  
  return (df)
}
   
# getDraw probability
    
getDraw = function(data) {
    draw = 0
    for (i in 1:length(data[[1]])) {
        draw = draw + data[i, i]
    }
    return (draw)
}
    
# getWin probability

getWin = function(data) {
    win = 0
    for (i in 1:length(data[[1]])) {
        j = 1
        #print("here")
        for (j in (1+i):length(data[[1]])-1) {
            if(j != i) {
                #print(paste0('data[', j, ',', i, ']'))
                #print(data[j, i])
                win = win + data[j, i]
            }
            
        }
    }
    return (win)
}
    
# getLoose probability

getLoose = function(data) {
    win = 0
    for (i in 1:length(data[[1]])) {
        j = 1
        for (j in 1:length(data[[1]])) {
            if(j != i & j < i) {
                #print(paste0('data[', j, ',', i, ']'))
                #print(data[j, i])
                win = win + data[j, i]
            }
            
        }
    }
    return (win)
}

## miscellenious functions

In [23]:
isNeutralAdjusted = function(country, opponent, game_name) {
  
  HOSTCOUNTRY = NULL
  
  if(game_name == 'World Cup 2010') {
    HOSTCOUNTRY = 'South Africa'
  } else if(game_name == 'World Cup 2014') {
    HOSTCOUNTRY = 'Brazil'
  } else if(game_name == 'World Cup 2018') {
    HOSTCOUNTRY = 'Russia'
  } else if(game_name == 'World Cup 2022') {
    HOSTCOUNTRY = 'Qatar'
  } 
  
  if (!is.null(HOSTCOUNTRY)) {
    venue = 'N'
    if(country != HOSTCOUNTRY & opponent != HOSTCOUNTRY) {
      return('N')
    } else if(country == HOSTCOUNTRY) {
      return('H')
    } else if(opponent == HOSTCOUNTRY) {
      return('A')
    }
  } else {
    return ('H/A')
  }
}
      
ternary = function(expression_1, expression_2, condition) {
  if(condition) {
    expression_1
  } else {
    expression_2
  }
}

## getGoals Function and its dependent functions

In [104]:
fifa_ranking1

Land,Rang,Punkte,fifa_date
<chr>,<chr>,<chr>,<date>
Spain,1,1588,2009-09-02
Netherlands,2,1376,2009-09-02
Germany,3,1186,2009-09-02
Italy,4,1186,2009-09-02
Russia,5,1129,2009-09-02
England,6,1127,2009-09-02
Argentina,7,1113,2009-09-02
Croatia,8,1101,2009-09-02
France,9,1040,2009-09-02


In [108]:
# getFifaRanking
getFifaRanking = function(country, date) {
  
#   print(fifa_ranking1)
    
  fifa_ranking1 %>%
    filter(Land == country) %>%
    filter(fifa_date == date) %>%
    pluck(2) -> temp
  #print(as.numeric(unlist(temp)))
  
  as.numeric(temp)
}

# test_date

test_date = function(date_param) {
  i = 1;
  
  is_lower = F
    
  date_list = unique(fifa_ranking1$fifa_date)
    
  date_list
  
  # loops through a date_list which we got from the fifa ranking dataset
  for(i in 1:length(date_list)) {
    # test if the supplied date_param is smaller then the fifa_date inside date_list
    # if it is, the loop will break, which means it stops and jumps out of the loop
    if(date_list[i] > date_param) {
      is_lower = F
      break;
    } else {
      is_lower = T
    }
  }
  # This is were the execution continous when we break out of the loop
  # This is also the place where we continue execution in case the for loop finishes through all iterations
  if(i != 1 & is_lower == F) {
    # if i does not stay 1 & a date value was found which exceeds the date_param then
    # it should always return the value at position i - 1
    return(date_list[i-1])
  } else if( i != 1 & is_lower == T) {
    # if i does not stay 1 & a date value was not found which exceeds the date_param then
    # it should always return the value at position i
    return(date_list[i])
  } else {
    # if i does stay 1 it should always return 1
    return(date_list[1])
  }
}

# get goals

getGoals = function(df, model, team, opponent, date) {
  # obtains the fifa date that comes before the date paramter
  fifa_date = test_date(date)
  
  dataFrame = data.frame(
    rank_opponent = getFifaRanking(opponent, fifa_date),
    Venue_H = ternary(1, 0, isNeutralAdjusted(team, opponent, 'World Cup 2018') == 'H'),
    Venue_N = ternary(1, 0, isNeutralAdjusted(team, opponent, 'World Cup 2018') == 'N'),
    Rank = getFifaRanking(team, fifa_date))
  
  lambda = predict(model, newdata = dataFrame, type = "response")
  return (rpois(1, lambda))
}

## getGroupPlaySummary

In [16]:
getGroupPlaySummary = function(df, model, team_A, team_B, team_C, team_D, date) {
  
  team_A_points = 0
  team_B_points = 0
  team_C_points = 0
  team_D_points = 0
  
  goal_difference_A = 0
  goal_difference_B = 0
  goal_difference_C = 0
  goal_difference_D = 0
  
  goals_A = 0
  goals_B = 0
  goals_C = 0
  goals_D = 0
  
  getResult = function(df, model, team, opponent) {
    
    team_goals = getGoals(df, model, team, opponent, date)
    
    opponent_goals = getGoals(df, model, opponent, team, date)
    
    return (c(team_goals, opponent_goals))
  }
  
  update_points = function(goals, team, opponent, team_points_param, opponent_points_param) {
    list_ = goals
    team_points = NULL
    opponent_points = NULL
    
    goals_team = list_[1]
    goals_opponent = list_[2]
    
    if(goals_team > goals_opponent) {
      team_points = team_points_param + 3
      opponent_points = opponent_points_param + 0
    } else if(goals_team < goals_opponent) {
      opponent_points = opponent_points_param + 3
      team_points = team_points_param + 0
    } else if(goals_team == goals_opponent) {
      team_points = team_points_param + 1
      opponent_points = opponent_points_param + 1
    }
    
    return (c(team_points, opponent_points))
  }
  
  update_goal_difference = function(goals, team, opponent, team_goal_difference_param, opponent_goal_difference_param) {
    list_ = goals
    team_goal_difference = NULL
    opponent_goal_difference = NULL
    
    goals_team = list_[1]
    goals_opponent = list_[2]
    
    team_goal_difference = 
      team_goal_difference_param +
      goals_team -
      goals_opponent
    
    opponent_goal_difference = 
      opponent_goal_difference_param +
      goals_opponent -
      goals_team
    
    return (c(team_goal_difference, opponent_goal_difference))
  }
  
  # team A - team B
  goals = getResult(df, model, team_A, team_B)
  points = update_points(goals, team_A, team_B, team_A_points, team_B_points)
  difference = update_goal_difference(goals, team_A, team_B, goal_difference_A, goal_difference_B)

  team_A_points = points[1]
  team_B_points = points[2]
  goal_difference_A = difference[1]
  goal_difference_B = difference[2]
  goals_A = goals_A + goals[1]
  goals_B = goals_B + goals[2]
  
  # team A - team C
  goals = getResult(df, model, team_A, team_C)
  points = update_points(goals, team_A, team_C, team_A_points, team_C_points)
  difference = update_goal_difference(goals, team_A, team_C, goal_difference_A, goal_difference_C)
  team_A_points = points[1]
  team_C_points = points[2]
  goal_difference_A = difference[1]
  goal_difference_C = difference[2]
  goals_A = goals_A + goals[1]
  goals_C = goals_C + goals[2]
  
  # team A - team D
  goals = getResult(df, model, team_A, team_D)
  points = update_points(goals, team_A, team_D, team_A_points, team_D_points)
  difference = update_goal_difference(goals, team_A, team_D, goal_difference_A, goal_difference_D)
  team_A_points = points[1]
  team_D_points = points[2]
  goal_difference_A = difference[1]
  goal_difference_D = difference[2]
  goals_A = goals_A + goals[1]
  goals_D = goals_D + goals[2]
  
  # team B - team C
  goals = getResult(df, model, team_B, team_C)
  points = update_points(goals, team_B, team_C, team_B_points, team_C_points)
  difference = update_goal_difference(goals, team_B, team_C, goal_difference_B, goal_difference_C)
  team_B_points = points[1]
  team_C_points = points[2]
  goal_difference_B = difference[1]
  goal_difference_C = difference[2]
  goals_B = goals_B + goals[1]
  goals_C = goals_C + goals[2]
  
  # team B - team D
  goals = getResult(df, model, team_B, team_D)
  points = update_points(goals, team_B, team_D, team_B_points, team_D_points)
  difference = update_goal_difference(goals, team_B, team_D, goal_difference_B, goal_difference_D)
  team_B_points = points[1]
  team_D_points = points[2]
  goal_difference_B = difference[1]
  goal_difference_D = difference[2]
  goals_B = goals_B + goals[1]
  goals_D = goals_D + goals[2]
  
  # team C - team D
  goals = getResult(df, model, team_B, team_D)
  points = update_points(goals, team_C, team_D, team_C_points, team_D_points)
  difference = update_goal_difference(goals, team_C, team_D, goal_difference_C, goal_difference_D)
  team_C_points = points[1]
  team_D_points = points[2]
  goal_difference_C = difference[1]
  goal_difference_D = difference[2]
  goals_C = goals_C + goals[1]
  goals_D = goals_D + goals[2]
  
  data.frame(
    Country=c(team_A, team_B, team_C, team_D),
    Points=c(team_A_points, team_B_points, team_C_points, team_D_points),
    Goal_difference=c(goal_difference_A, goal_difference_B, goal_difference_C, goal_difference_D),
    Goals_scored=c(goals_A, goals_B, goals_C, goals_D)
  ) -> ranking
  
  ranking %>%
    group_by(Points) %>%
    arrange(desc(Goal_difference), desc(Goals_scored), .by_group = T) %>%
    arrange(desc(Points)) -> ranking
  
  return (ranking)
}

In [57]:
#getGroupPlaySummary(test, model, 'Netherlands', 'Senegal', 'Ecuador', 'Qatar', '2022-11-29')

## getPlaySummary and its dependent functions

In [48]:
penaltyShootout = function() {
  goals_team_A = 0
  goals_team_B = 0
  possibilities = c('goal', 'no_goal')
  sample(possibilities, size=1)
  
  for(i in 1:5) {
    if(sample(possibilities, size=1) == 'goal') {
      goals_team_A = goals_team_A + 1
    }
    
    if(sample(possibilities, size=1) == 'no_goal') {
      goals_team_B = goals_team_B + 1
    }
  }
  
  return (c(
    result_team_A = goals_team_A,
    result_team_B = goals_team_B
  )) 
}

getPlaySummary = function(df, model, team_A, team_B, date) {
  
  result = c()
  country = c()
  opponent = c()
  
  team_A_goals = 0
  team_B_goals = 0
  
  getResult = function(df, model, team, opponent, date) {
    
    team_goals = getGoals(df, model, team, opponent, date)
    
    opponent_goals = getGoals(df, model, opponent, team, date)
    
    return (c('team'=team_goals, 'opponent'=opponent_goals))
  }
  
  # team A - team B
  goals = getResult(df, model, team_A, team_B, date)
  
  winning_team = NULL
  value = 0
  
  penaltyGoals = list(
    result_team_A = 0,
    result_team_B = 0
  )
  
  team_A_goals = team_A_goals + as.numeric(unname(goals['team']))
  team_B_goals = team_B_goals + as.numeric(unname(goals['opponent']))
  
  if(team_A_goals > team_B_goals) {
    value = 1
    winning_team = team_A
  } else if (team_A_goals < team_B_goals) {
    value = 2
    winning_team = team_B
  } else if (team_A_goals == team_B_goals) {
    value = 3
    penaltyGoals = penaltyShootout()
    team_A_goals = team_A_goals + as.numeric(unname(penaltyGoals['result_team_A']))
    team_B_goals = team_B_goals + as.numeric(unname(penaltyGoals['result_team_B']))
    
    while (team_A_goals == team_B_goals) {
      penaltyGoals = penaltyShootout()
      team_A_goals = team_A_goals + as.numeric(unname(penaltyGoals['result_team_A']))
      team_B_goals = team_B_goals + as.numeric(unname(penaltyGoals['result_team_B']))
    }
    
    if(team_A_goals > team_B_goals) {
      winning_team = team_A
    } else {
      winning_team = team_B
    }
  }

  return (winning_team)
}

In [58]:
#getPlaySummary(test, model, 'Netherlands', 'Senegal', '2022-11-20')

## Import preprocessed data

In [128]:
data = read.csv('https://raw.githubusercontent.com/denisnaumov0506/FootballAnalyticsProject/master/data/world_cup%26friendlies_football_en.csv', header= T, sep= ';')
fifa_ranking1 = read.csv("https://raw.githubusercontent.com/denisnaumov0506/FootballAnalyticsProject/master/Whole_FIFA_Ranking.csv", header = TRUE, sep = ",", stringsAsFactors=FALSE, fileEncoding="latin1")

In [129]:
# train test

fifa_ranking1

data %>%
    filter(game_name != 'World Cup 2022' 
           & date_of_play < ('2022-11-20')
          ) -> train

data %>%
    filter(game_name == 'World Cup 2022') -> test

Land,Rang,Punkte,fifa_date
<chr>,<int>,<chr>,<chr>
Spain,1,1588,2009-09-02
Netherlands,2,1376,2009-09-02
Germany,3,1186,2009-09-02
Italy,4,1186,2009-09-02
Russia,5,1129,2009-09-02
England,6,1127,2009-09-02
Argentina,7,1113,2009-09-02
Croatia,8,1101,2009-09-02
France,9,1040,2009-09-02


In [5]:
train

country,fifa_date,Matchday,Time,Venue,Opponent,System_of_play,Attendance,result_left,result_right,game_name,goals_country,goals_enemy,date_of_play,rank_opponent,Rank,Venue_A,Venue_H,Venue_N
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<chr>,<int>,<int>,<chr>,<int>,<int>,<int>,<int>,<int>
Argentina,2009-12-16,NA,2:00 AM,H,Costa Rica,4-4-2 Diamond,19.000,3,2,International Friendlies,3,2,2010-01-27,44,8,0,1,0
Argentina,2010-02-03,NA,2:00 AM,H,Jamaica,4-4-2 Diamond,20.000,2,1,International Friendlies,2,1,2010-02-11,78,8,0,1,0
Argentina,2010-03-03,NA,8:45 PM,A,Germany,4-4-2 double 6,65.152,0,1,International Friendlies,1,0,2010-03-03,5,9,1,0,0
Argentina,2010-04-28,NA,11:00 PM,H,Haiti,4-4-2 Diamond,,4,0,International Friendlies,4,0,2010-05-05,87,7,0,1,0
Argentina,2010-04-28,NA,9:00 PM,H,Canada,4-4-2 Diamond,52.000,5,0,International Friendlies,5,0,2010-05-24,63,7,0,1,0
Argentina,2010-05-26,Group B,4:00 PM,N,Nigeria,4-4-2 Diamond,55.686,1,0,World Cup 2010,1,0,2010-06-12,21,7,0,0,1
Argentina,2010-05-26,Group B,1:30 PM,N,South Korea,4-4-2 Diamond,82.174,4,1,World Cup 2010,4,1,2010-06-17,47,7,0,0,1
Argentina,2010-05-26,Group B,8:30 PM,N,Greece,4-4-2 Diamond,38.891,0,2,World Cup 2010,2,0,2010-06-22,13,7,0,0,1
Argentina,2010-05-26,Round of 16,8:30 PM,N,Mexico,4-4-2 Diamond,84.377,3,1,World Cup 2010,3,1,2010-06-27,17,7,0,0,1


In [6]:
#train
data

model = 
    glm(
        goals_country ~ Rank + rank_opponent + Venue_H + Venue_N,
        data,
        family = poisson(link = 'log')
       )
summary(model)

country,fifa_date,Matchday,Time,Venue,Opponent,System_of_play,Attendance,result_left,result_right,game_name,goals_country,goals_enemy,date_of_play,rank_opponent,Rank,Venue_A,Venue_H,Venue_N
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<chr>,<int>,<int>,<chr>,<int>,<int>,<int>,<int>,<int>
Argentina,2009-12-16,NA,2:00 AM,H,Costa Rica,4-4-2 Diamond,19.000,3,2,International Friendlies,3,2,2010-01-27,44,8,0,1,0
Argentina,2010-02-03,NA,2:00 AM,H,Jamaica,4-4-2 Diamond,20.000,2,1,International Friendlies,2,1,2010-02-11,78,8,0,1,0
Argentina,2010-03-03,NA,8:45 PM,A,Germany,4-4-2 double 6,65.152,0,1,International Friendlies,1,0,2010-03-03,5,9,1,0,0
Argentina,2010-04-28,NA,11:00 PM,H,Haiti,4-4-2 Diamond,,4,0,International Friendlies,4,0,2010-05-05,87,7,0,1,0
Argentina,2010-04-28,NA,9:00 PM,H,Canada,4-4-2 Diamond,52.000,5,0,International Friendlies,5,0,2010-05-24,63,7,0,1,0
Argentina,2010-05-26,Group B,4:00 PM,N,Nigeria,4-4-2 Diamond,55.686,1,0,World Cup 2010,1,0,2010-06-12,21,7,0,0,1
Argentina,2010-05-26,Group B,1:30 PM,N,South Korea,4-4-2 Diamond,82.174,4,1,World Cup 2010,4,1,2010-06-17,47,7,0,0,1
Argentina,2010-05-26,Group B,8:30 PM,N,Greece,4-4-2 Diamond,38.891,0,2,World Cup 2010,2,0,2010-06-22,13,7,0,0,1
Argentina,2010-05-26,Round of 16,8:30 PM,N,Mexico,4-4-2 Diamond,84.377,3,1,World Cup 2010,3,1,2010-06-27,17,7,0,0,1



Call:
glm(formula = goals_country ~ Rank + rank_opponent + Venue_H + 
    Venue_N, family = poisson(link = "log"), data = data)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.8266  -1.1876  -0.1969   0.5617   3.7318  

Coefficients:
                Estimate Std. Error z value Pr(>|z|)    
(Intercept)    0.1958458  0.0399740   4.899 9.62e-07 ***
Rank          -0.0088665  0.0007454 -11.895  < 2e-16 ***
rank_opponent  0.0071007  0.0004399  16.141  < 2e-16 ***
Venue_H        0.2695579  0.0377726   7.136 9.58e-13 ***
Venue_N        0.1828919  0.0612309   2.987  0.00282 ** 
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for poisson family taken to be 1)

    Null deviance: 3201.9  on 2222  degrees of freedom
Residual deviance: 2807.4  on 2218  degrees of freedom
  (176 observations deleted due to missingness)
AIC: 7001.7

Number of Fisher Scoring iterations: 5


## Simulation functions

In [112]:
simulateGroupPlay = function(iter, df, model, team1, team2, team3, team4, date) {
  first_place1 = 0
  second_place1 = 0
  third_place1 = 0
  fourth_place1 = 0
  
  first_place2 = 0
  second_place2 = 0
  third_place2 = 0
  fourth_place2 = 0
  
  first_place3 = 0
  second_place3 = 0
  third_place3 = 0
  fourth_place3 = 0
  
  first_place4 = 0
  second_place4 = 0
  third_place4 = 0
  fourth_place4 = 0
  
  n = iter
  
  for(i in 1:n) {
    print(i)
    #summary = getGroupPlaySummary(test, model, 'France', 'Denmark', 'Peru', 'Australia')
    summary = getGroupPlaySummary(df, model, team1, team2, team3, team4, date)
    
    if(summary[1,1] == team1) {
      first_place1 = first_place1 + 1
    } else if (summary[2,1] == team1) {
      second_place1 = second_place1 + 1
    } else if (summary[3,1] == team1) {
      third_place1 = third_place1 + 1
    } else if (summary[4,1] == team1) {
      fourth_place1 = fourth_place1 + 1
    }
    
    if(summary[1,1] == team2) {
      first_place2 = first_place2 + 1
    } else if (summary[2,1] == team2) {
      second_place2 = second_place2 + 1
    } else if (summary[3,1] == team2) {
      third_place2 = third_place2 + 1
    } else if (summary[4,1] == team2) {
      fourth_place2 = fourth_place2 + 1
    }
    
    if(summary[1,1] == team3) {
      first_place3 = first_place3 + 1
    } else if (summary[2,1] == team3) {
      second_place3 = second_place3 + 1
    } else if (summary[3,1] == team3) {
      third_place3 = third_place3 + 1
    } else if (summary[4,1] == team3) {
      fourth_place3 = fourth_place3 + 1
    }
    
    if(summary[1,1] == team4) {
      first_place4 = first_place4 + 1
    } else if (summary[2,1] == team4) {
      second_place4 = second_place4 + 1
    } else if (summary[3,1] == team4) {
      third_place4 = third_place4 + 1
    } else if (summary[4,1] == team4) {
      fourth_place4 = fourth_place4 + 1
    }
    
  }
  
  df = data.frame(
    team1 = c(first_place1/n, second_place1/n, third_place1/n, fourth_place1/n),
    team2 = c(first_place2/n, second_place2/n, third_place2/n, fourth_place2/n),
    team3 = c(first_place3/n, second_place3/n, third_place3/n, fourth_place3/n),
    team4 = c(first_place4/n, second_place4/n, third_place4/n, fourth_place4/n)
    )
  
  df = setNames(df, c(team1, team2, team3, team4))
  
  return(df)
}

simulatePlay = function(iter, team_1, team_2, date) {
  
  vector = c()
  
  for(i in 1:iter) {
    print(i)
    result = getPlaySummary(test, model, team_A = team_1, team_B = team_2, date)
    vector = append(vector, result)
  }
  
  total = 0
  
  for(item in vector) {
    if(item == team_1) {
      total = total + 1
    }
  }
  
  return(total/iter)
}

simulateFinalPlays = function(iter) {
  
  vec = c(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
  
  for(i in 1:iter) {
    print(i)
    # Round 16
    playSum = getPlaySummary(
      df, model,
      'France', # A1
      'Argentina', # B2
      '2018-06-30'
    )
    
    winnerA1_B2 = playSum
    
    playSum = getPlaySummary(
      df, model,
      'Uruguay', # C1
      'Portugal', # D2
      '2018-07-01'
    )
    
    winnerC1_D2 = playSum
    
    playSum = getPlaySummary(
      df, model, 
      'Spain', #E1
      'Russia', #F2
      '2018-07-02'
    )
    
    winnerE1_F2 = playSum
    
    playSum = getPlaySummary(
      df, model, 
      'Croatia', #G1
      'Denmark', #H2
      '2018-07-03'
    )
    
    winnerG1_H2 = playSum
    
    playSum = getPlaySummary(
      df, model,
      'Brazil', # A2
      'Mexico', # B1
      '2018-06-30'
    )
    
    winnerB1_A2 = playSum
    
    playSum = getPlaySummary(
      df, model,
      'Belgium', # C2
      'Japan', # D1
      '2018-07-01'
    )
    
    winnerD1_C2 = playSum
    
    playSum = getPlaySummary(
      df, model,
      'Sweden', #E2
      'Switzerland', #F1
      '2018-07-02'
    )
    
    winnerF1_E2 = playSum
    
    playSum = getPlaySummary(
      df, model, 
      'Colombia', #G2
      'England', #H1
      '2018-07-03'
    )
    
    winnerH1_G2 = playSum
    
    # Viertelfinal
    playSum = getPlaySummary(
      df, model,
      winnerA1_B2,
      winnerC1_D2,
      '2018-07-06'
    )
    
    winnerQFA = playSum
    
    playSum = getPlaySummary(
      df, model,
      winnerE1_F2,
      winnerG1_H2,
      '2018-07-06'
    )
    
    winnerQFB = playSum
    
    playSum = getPlaySummary(
      df, model,
      winnerB1_A2,
      winnerD1_C2,
      '2018-07-07'
    )
    
    winnerQFC = playSum
    
    playSum = getPlaySummary(
      df, model,
      winnerF1_E2,
      winnerH1_G2,
      '2018-07-07'
    )
    
    winnerQFD = playSum
    
    # Semi final
    playSum = getPlaySummary(
      df, model,
      winnerQFA,
      winnerQFB,
      '2018-07-10'
    )
    
    winnerSFA = playSum
    
    playSum = getPlaySummary(
      df, model,
      winnerQFC,
      winnerQFD,
      '2018-07-11'
    )
    
    winnerSFB = playSum
    
    # final
    playSum = getPlaySummary(
      df, model,
      winnerSFA,
      winnerSFB,
      '2018-07-15'
    )
    
    winnerF = playSum
    #print(sprintf("The winner of the world cup is: %s", winnerF))
    if(winnerF == 'France') {
      #france = france + 1
      vec[1] = vec[1] + 1
    } else if(winnerF == 'Argentina') {
      #argentina = argentina + 1
      vec[2] = vec[2] + 1
    } else if(winnerF == 'Uruguay') {
      #uruguay = uruguay + 1
      vec[3] = vec[3] + 1
    } else if(winnerF == 'Portugal') {
      #portugal = portugal + 1
      vec[4] = vec[4] + 1
    } else if(winnerF == 'Spain') {
      #spain = spain + 1
      vec[5] = vec[5] + 1
    } else if(winnerF == 'Russia') {
      #russia = russia + 1
      vec[6] = vec[6] + 1
    } else if(winnerF == 'Croatia') {
      #croatia = croatia + 1
      vec[7] = vec[7] + 1
    } else if(winnerF == 'Denmark') {
      #denmark = denmark + 1
      vec[8] = vec[8] + 1
    } else if(winnerF == 'Brazil') {
      #brazil = brazil + 1
      vec[9] = vec[9] + 1
    } else if(winnerF == 'Mexico') {
      #mexico = mexico + 1
      vec[10] = vec[10] + 1
    } else if(winnerF == 'Belgium') {
      #belgium = belgium + 1
      vec[11] = vec[11] + 1
    } else if(winnerF == 'Japan') {
      #japan = japan + 1
      vec[12] = vec[12] + 1
    } else if(winnerF == 'Sweden') {
      #sweden = sweden + 1
      vec[13] = vec[13] + 1
    } else if(winnerF == 'Switzerland') {
      #switzerland = switzerland + 1
      vec[14] = vec[14] + 1
    } else if(winnerF == 'Colombia') {
      #colombia = colombia + 1
      vec[15] = vec[15] + 1
    } else if(winnerF == 'England') {
      #england = england + 1
      vec[16] = vec[16] + 1
    }
  }
  df = data.frame(
    France = vec[1]/iter,
    Argentina = vec[2]/iter,
    Uruguay = vec[3]/iter,
    Portugal = vec[4]/iter,
    Spain = vec[5]/iter,
    Russia = vec[6]/iter,
    Croatia = vec[7]/iter,
    Denmark = vec[8]/iter,
    Brazil = vec[9]/iter,
    Mexico = vec[10]/iter,
    Belgium = vec[11]/iter,
    Japan = vec[12]/iter,
    Sweden = vec[13]/iter,
    Switzerland = vec[14]/iter,
    Colombia = vec[15]/iter,
    England = vec[16]/iter
  )
  
  return(df)
}

In [139]:
set.seed(1000)

groupA = simulateGroupPlay(iter = 1000, test, model, 'Netherlands', 'Senegal', 'Ecuador', 'Qatar', '2022-11-29')
groupB = simulateGroupPlay(iter = 1000, test, model, 'England', 'United States', 'Iran', 'Wales', '2022-11-29')
groupC = simulateGroupPlay(iter = 1000, test, model, 'Argentina', 'Poland', 'Saudi Arabia', 'Mexico', '2022-11-30')
groupD = simulateGroupPlay(iter = 1000, test, model, 'France', 'Australia', 'Denmark', 'Tunisia', '2022-11-30')
groupE = simulateGroupPlay(iter = 1000, test, model, 'Germany', 'Spain', 'Japan', 'Costa Rica', '2022-12-01')
groupF = simulateGroupPlay(iter = 1000, test, model, 'Belgium', 'Canada', 'Croatia', 'Morocco', '2022-12-01')
groupG = simulateGroupPlay(iter = 1000, test, model, 'Switzerland', 'Cameroon', 'Serbia', 'Brazil', '2022-12-02')
groupH = simulateGroupPlay(iter = 1000, test, model, 'Portugal', 'Ghana', 'Uruguay', 'South Korea', '2022-12-02')

[1] 1
[1] 2
[1] 3
[1] 4
[1] 5
[1] 6
[1] 7
[1] 8
[1] 9
[1] 10
[1] 11
[1] 12
[1] 13
[1] 14
[1] 15
[1] 16
[1] 17
[1] 18
[1] 19
[1] 20
[1] 21
[1] 22
[1] 23
[1] 24
[1] 25
[1] 26
[1] 27
[1] 28
[1] 29
[1] 30
[1] 31
[1] 32
[1] 33
[1] 34
[1] 35
[1] 36
[1] 37
[1] 38
[1] 39
[1] 40
[1] 41
[1] 42
[1] 43
[1] 44
[1] 45
[1] 46
[1] 47
[1] 48
[1] 49
[1] 50
[1] 51
[1] 52
[1] 53
[1] 54
[1] 55
[1] 56
[1] 57
[1] 58
[1] 59
[1] 60
[1] 61
[1] 62
[1] 63
[1] 64
[1] 65
[1] 66
[1] 67
[1] 68
[1] 69
[1] 70
[1] 71
[1] 72
[1] 73
[1] 74
[1] 75
[1] 76
[1] 77
[1] 78
[1] 79
[1] 80
[1] 81
[1] 82
[1] 83
[1] 84
[1] 85
[1] 86
[1] 87
[1] 88
[1] 89
[1] 90
[1] 91
[1] 92
[1] 93
[1] 94
[1] 95
[1] 96
[1] 97
[1] 98
[1] 99
[1] 100
[1] 101
[1] 102
[1] 103
[1] 104
[1] 105
[1] 106
[1] 107
[1] 108
[1] 109
[1] 110
[1] 111
[1] 112
[1] 113
[1] 114
[1] 115
[1] 116
[1] 117
[1] 118
[1] 119
[1] 120
[1] 121
[1] 122
[1] 123
[1] 124
[1] 125
[1] 126
[1] 127
[1] 128
[1] 129
[1] 130
[1] 131
[1] 132
[1] 133
[1] 134
[1] 135
[1] 136
[1] 137
[1] 138
[1] 